# Noise Corrector CNN to get CR images without noise
---

---

In [1]:
# This is just a function to allow toggleing code cells that are too long for good
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

In [2]:
import torch #should be installed by default in any colab notebook
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import json
import os
import pandas as pd
import h5py
from time import time
from datetime import datetime
from IPython import display as display_IPython

assert torch.cuda.is_available(), "GPU is not enabled"

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the functions and routines for the DL
### Define the model and its constructor

In [3]:
class Noise_Corrector(nn.Module):
    def __init__(self, S0=2*302+1, S1=2*290+1, S2=2*250+1, S3=2*200+1,
                 feats_S1=10, feats_S2=10, feats_S3=20, feats_S0=10,
                 dropout_p=0.1
                ): 
       
        super(Noise_Corrector, self).__init__()
        self.Ss = [S0, S1, S2, S3]
        self.feats = [feats_S0, feats_S1, feats_S2, feats_S3]
        # in is [batch_size, 1, S0, S0]
        self.conv_S01 = nn.Conv2d(in_channels=1, out_channels=feats_S1, 
                               kernel_size = S0-S1+1, bias=True) 
        # out conv_S01 [batch_size, feats_S1, S1, S1]
        self.conv_S12 = nn.Conv2d(in_channels=feats_S1, out_channels=feats_S2, 
                               kernel_size = S1-S2+1, bias=True) 
        # out conv_S12 [batch_size, feats_S2, S2, S2]
        self.conv_S23 = nn.Conv2d(in_channels=feats_S2, out_channels=feats_S3, 
                               kernel_size = S2-S3+1, bias=True) 
        # out conv_S23 [batch_size, feats_S3, S3, S3]
        
        self.deConv_S32 = torch.nn.ConvTranspose2d(in_channels=feats_S3, out_channels=feats_S2, 
                                                kernel_size = S2-S3+1)
        # out deConv_S32+memory [batch_size, 2*feats_S2, S2, S2]
        self.deConv_S21 = torch.nn.ConvTranspose2d(in_channels=2*feats_S2, out_channels=feats_S1, 
                                                kernel_size = S1-S2+1)
        # out deConv_S21+memory [batch_size, 2*feats_S1, S1, S1]
        self.deConv_S10 = torch.nn.ConvTranspose2d(in_channels=2*feats_S1, out_channels=feats_S0, 
                                                kernel_size = S0-S1+1)

        # out conv_S01 [batch_size, feats_S1, S1, S1]
        self.conv_S00 = nn.Conv2d(in_channels=feats_S0+1, out_channels=1, 
                               kernel_size = 1, bias=True) 
        
        
        self.dropout = nn.Dropout(p=dropout_p, inplace=False)
        self.relu = torch.nn.functional.leaky_relu

        self.batchNorm = nn.BatchNorm2d(num_features=feats_S3)

    def forward(self, x): # [batch_size, 2X+1, 2X+1] or [batch_size, 1, 2X+1, 2X+1] Already Normalized!
        x = x.view(x.shape[0], 1, x.shape[-2], x.shape[-1]).float() # [batch_size, 1, 2X+1, 2X+1]
        # Normalize to unity the float image
        #x = x/x.amax(dim=(2,3), keepdim=True)[0] # [batch_size, 1, 2X+1, 2X+1]
        
        # Conv layers
        s1 = self.relu(self.conv_S01(x)) # [batch_size, feats_S1, S1, S1]
        s2 = self.dropout( self.relu(self.conv_S12(s1)) ) # [batch_size, feats_S2, S2, S2]
        s3 = self.batchNorm( self.relu(self.conv_S23(s2)) ) # [batch_size, feats_S3, S3, S3]
        # deConv layers
        s2 = torch.cat((s2, self.relu(self.deConv_S32(s3))), 1) #[batch_size, 2*feats_S2, S2, S2]
        s1 = torch.cat((s1, self.relu(self.deConv_S21( self.dropout(s2) )) ), 1) #[batch_size, 2*feats_S1, S1, S1]
        x = torch.cat((x, self.relu(self.deConv_S10(s1))), 1) #[batch_size, 1+feats_S0, S0, S0]
        # Conv layer
        return self.conv_S00(x) # [batch_size, 1, S0, S0]

    
    def print_shapes(self, batch_size=10):
        x = torch.ones((batch_size, 1, self.Ss[0], self.Ss[0])).to(device)
        print(f"Initial shape {x.shape}")
        s1 = self.relu(self.conv_S01(x)) # [batch_size, feats_S1, S1, S1]
        print(f"Conv01 shape {s1.shape} should be [{batch_size},{self.feats[1]},{self.Ss[1]}, {self.Ss[1]}]")
        s2 = self.dropout( self.relu(self.conv_S12(s1)) ) # [batch_size, feats_S2, S2, S2]
        print(f"Conv12 shape {s2.shape} should be [{batch_size},{self.feats[2]},{self.Ss[2]}, {self.Ss[2]}]")
        s3 = self.batchNorm( self.relu(self.conv_S23(s2)) ) # [batch_size, feats_S3, S3, S3]
        print(f"Conv23 shape {s3.shape} should be [{batch_size},{self.feats[3]},{self.Ss[3]}, {self.Ss[3]}]")

        s2 = torch.cat((s2, self.relu(self.deConv_S32(s3))), 1) #[batch_size, 2*feats_S2, S2, S2]
        print(f"DeConv32+mem shape {s2.shape} should be [{batch_size},{2*self.feats[2]},{self.Ss[2]}, {self.Ss[2]}]")
        s1 = torch.cat((s1, self.relu(self.deConv_S21( self.dropout(s2) )) ), 1) #[batch_size, 2*feats_S1, S1, S1]
        print(f"DeConv21+mem shape {s1.shape} should be [{batch_size},{2*self.feats[1]},{self.Ss[1]}, {self.Ss[1]}]")
        x = torch.cat((x, self.relu(self.deConv_S10(s1))), 1) #[batch_size, 1+feats_S0, S0, S0]
        print(f"DeConv10+mem shape {x.shape} should be [{batch_size},{1+self.feats[0]},{self.Ss[0]}, {self.Ss[0]}]")
        x= self.conv_S00(x) # [batch_size, 1, S0, S0]
        print(f"Conv00 shape {x.shape} should be [{batch_size}, 1, {self.Ss[0]}, {self.Ss[0]}]")

    

In [4]:
hide_toggle(for_next=True)


In [5]:
# subroutine to count number of parameters in the model
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.numel()
    return np

### The routines to validate and train

In [6]:
hide_toggle(for_next=True)


In [7]:
@torch.no_grad()  # prevent this function from computing gradients 
def validate_epoch(criterion, model, dataloader, per_epoch_use_max_batches=None): #show_confusion_matrix = False):
    if per_epoch_use_max_batches is None:
        per_epoch_use_max_batches = len(dataloader)
    val_loss = 0
    max_abs_error = torch.Tensor([0]).to(device)
    mean_abs_error = 0
    preds = torch.Tensor().to(device)
    targets = torch.Tensor().to(device)

    model.eval() # disable the dropout, among others

    for batch_id in range(len(dataloader)):  
        data, target = dataloader[batch_id] # dataloader sends them to device already   
        
        # images to float
        data = data.view(data.shape[0], 1, data.shape[-2], data.shape[-1]).float()
        target = target.view(data.shape[0], 1, data.shape[-2], data.shape[-1]).float()
        # normalize the data and target
        data = data/data.amax(dim=(-2,-1), keepdim=True)[0] # [batch_size, 1, 2X+1, 2X+1]
        target = target/target.amax(dim=(-2,-1), keepdim=True)[0]
        prediction = model(data) # data is [batch_size, embedding_dim]
        loss = criterion(prediction, target)
        val_loss += loss.item()             
        max_abs_error = torch.maximum(torch.max(torch.abs(prediction-target), 0).values, max_abs_error)
        mean_abs_error += torch.sum(torch.mean(torch.abs(prediction-target), (-2,-1)), 0)
        if batch_id % per_epoch_use_max_batches == per_epoch_use_max_batches-1:
            break
    val_loss /= min(len(dataloader), per_epoch_use_max_batches)
    mean_abs_error /= min(len(dataloader), per_epoch_use_max_batches)
    #accuracy = 100. * correct / len(loader.dataset)
    print(f'\nValidation set: Average loss: {val_loss:.4f}, Average Abs Error: {np.array(mean_abs_error.cpu())}, Maximum Abs Error: {np.array(max_abs_error.cpu())} \n')

    #if show_confusion_matrix:
    #    visualize_confusion_matrix(preds.to(torch.device('cpu')), targets.to(torch.device('cpu')))

    return val_loss


def train_epoch(epoch, criterion, model, optimizer, dataloader, print_loss_every_batches=20,
                optimizer_step_every_batches=1, per_epoch_use_max_batches=None):
    if per_epoch_use_max_batches is None:
        per_epoch_use_max_batches = len(dataloader)
        
    total_loss = 0.0

    model.train()

    optimizer.zero_grad()
    #t = time()
    random_indices = np.random.choice(range(len(dataloader)), per_epoch_use_max_batches, replace=False)
    for batch_id, idx in enumerate(random_indices):  
        data, target = dataloader[idx] # dataloader sends them to device already
        #data, target = data.to(device), target.to(device)
        
        # images to float
        data = data.view(data.shape[0], 1, data.shape[-2], data.shape[-1]).float()
        target = target.view(data.shape[0], 1, data.shape[-2], data.shape[-1]).float()
        # normalize the data and target
        data = data/data.amax(dim=(-2,-1), keepdim=True)[0] # [batch_size, 1, 2X+1, 2X+1]
        target = target/target.amax(dim=(-2,-1), keepdim=True)[0]

        prediction = model(data) # data is [batch_size, embedding_dim]
        loss = criterion(prediction, target)
        loss.backward()
        
        if batch_id % optimizer_step_every_batches==optimizer_step_every_batches-1:
            optimizer.step()
            optimizer.zero_grad()
        # print loss every N batches
        if batch_id % print_loss_every_batches == print_loss_every_batches-1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_id+1) * len(data), len(dataloader)*dataloader.batch_size,
                100*(batch_id+1)*len(data) / (len(dataloader)*dataloader.batch_size), loss.item()))

        #if batch_id % per_epoch_use_max_batches == per_epoch_use_max_batches-1:
        #    break


        total_loss += loss.item()  #.item() is very important here
        # In order to avoid having total_loss as a tensor in the gpu
        #t = time()

    return total_loss / min(len(dataloader), per_epoch_use_max_batches)

### The full training loop

In [8]:
hide_toggle(for_next=True)


In [9]:
def full_training_loop(model, criterion, optimizer, train_loader, test_loader, epochs=10,
                       print_loss_every_batches=20, validate_every_epochs=2, optimizer_step_every_batches=1,
                      per_epoch_use_max_train_batches=None, per_epoch_use_max_test_batches=None,
                      image_path=None, save_model_every_epochs=1, model_path=None, best_model_path=None):
    losses = {"train": [], "val": []}
    %matplotlib inline
    for epoch in range(epochs):

        train_loss = train_epoch(epoch, criterion, model, optimizer, train_loader,
                                 print_loss_every_batches=print_loss_every_batches,
                                optimizer_step_every_batches=optimizer_step_every_batches,
                                per_epoch_use_max_batches=per_epoch_use_max_train_batches)
        if epoch%validate_every_epochs==0 and epoch!=0:
            val_loss = validate_epoch(criterion, model, test_loader, per_epoch_use_max_test_batches)
        else:
            try:
                val_loss = losses["val"][-1]
            except:
                val_loss = train_loss
        if epoch and train_loss<=min(losses["train"]) and best_model_path:
            torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            }, best_model_path)
        
        losses["train"].append(train_loss)
        losses["val"].append(val_loss)        
        plt.plot(losses["train"], label="training loss")
        plt.plot(losses["val"], label="validation loss")
        plt.yscale('log')
        plt.legend()
        if image_path is not None:
            plt.savefig(image_path)
            plt.clf()
        else:
            display_IPython.clear_output(wait=True)
            plt.pause(0.001)
            plt.show()
        if epoch % save_model_every_epochs==save_model_every_epochs-1 and model_path:
            torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            }, model_path)

    return losses

### The Dataset class and Data Sampler

In [10]:
hide_toggle(for_next=True)

In [11]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader

class Noisy_Non_Noisy_Image_Dataloader(Dataset):
    def __init__(self, GT_file_path, h5f_full_path):
        self.df_GTs = pd.DataFrame.from_dict(json.load(open(GT_file_path))) 
        # Que te linkee el nombre de la entrada del h5f (un index del batch) con los ground truth phis

        self.h5f = h5py.File(f"{h5f_full_path}", 'r')
        self.num_batches = len(self.h5f)
        shape = np.array(self.h5f[ str(self.df_GTs['ID'][0]) ]).shape

        self.batch_size = shape[0]//2
        self.image_size = shape[1:]
        print(f"There are {self.batch_size} images per batch\nwith {self.image_size} size images\nand {self.num_batches} batches in total")
        print(f"A total of {self.batch_size*self.num_batches} images with their GTs (denoised versions).")

    def __len__(self):
        return self.num_batches

    def __getitem__(self, idx):
        data_and_gt = torch.tensor(np.array(self.h5f[str(idx)]), device=device, dtype=torch.float32).unsqueeze(1) 
        # [2*batch_size, 1, 2X+1, 2X1]                                          en h5f son uint8
        return data_and_gt[:self.batch_size], data_and_gt[self.batch_size:]

In [12]:
class denoise_criterion:
    def __init__(self, general_mse_over_base_mse, saturation_threshold):
        self.mse = nn.MSELoss()
        self.w  = general_mse_over_base_mse
        self.saturation_threshold = torch.tensor(saturation_threshold, device=device, dtype=torch.float32)

    def compute_loss(self, denoised, gt):
        denoised_sat = torch.where(denoised<self.saturation_threshold,
                                   denoised, self.saturation_threshold)/self.saturation_threshold
        gt_sat = torch.where(gt<self.saturation_threshold,
                                   gt, self.saturation_threshold)/self.saturation_threshold

        return self.w*self.mse(denoised, gt)+(1-self.w)*( self.mse(denoised_sat, gt_sat) )
    # si no funciona bien probar de ahcer una metrica aprate d ela mse ke pondere las diferencias por la media de las intensidades
    # uno sobre la media mas bien
    
class simple_denoise_criterion:
    def __init__(self, l2_vs_l1):
        self.mse = nn.MSELoss()
        self.l1 = nn.L1Loss()
        self.w = l2_vs_l1
        
    def compute_loss(denoised, gt):
        return self.w*self.mse(denoised, gt)+(1-self.w)*self.l1(denoised, gt)


---
# Initialize the dataset and sampler (choose the number of batches per epoch, and their length) and fix the artificial noise hyperparameters

Note that since in each epoch the dataset shown to the model will be random, we can use the same dataset as a validation set.

In [13]:
dataset_path = "/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/Noisy_Non_Noisy_Different_Angles"
GT_file_path_train = f"{dataset_path}/TRAIN/GROUND_TRUTHS.json"
images_h5_path_train = f"{dataset_path}/TRAIN/Noisy_Non_Noisy_Images_Dataset.h5" 
GT_file_path_test = f"{dataset_path}/TRAIN/GROUND_TRUTHS.json"
images_h5_path_test = f"{dataset_path}/TRAIN/Noisy_Non_Noisy_Images_Dataset.h5" 

save_stuff_path = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/Denoiser/"

total_epochs = 10000
validate_every_epochs = 20
optimizer_step_every_batches = 7
per_epoch_use_max_train_batches= 21
per_epoch_use_max_test_batches=3
save_model_every_epochs = 1
torch.manual_seed(681)

general_mse_over_base_mse = 0.8
saturation_threshold = 0.1

exp_name='INTENTO2_Noise_Corrector_Scratch_Network_general_MSE'


In [14]:
training_data = Noisy_Non_Noisy_Image_Dataloader(GT_file_path_train, images_h5_path_train)
#test_data = Noisy_Non_Noisy_Image_Dataloader(GT_file_path_test, images_h5_path_test)

There are 10 images per batch
with (605, 605) size images
and 20000 batches in total
A total of 200000 images with their GTs (denoised versions).


# Fix the Hyperparameters and Initialize the Model and the Optimizer

In [15]:
X=302
S0=2*X+1
S1=2*220+1
S2=2*180+1
S3=2*120+1
feats_S1=5
feats_S2=7
feats_S3=10
feats_S0=10
dropout_p=0.2

In [16]:
model = Noise_Corrector(S0=S0, S1=S1, S2=S2, S3=S3,
                 feats_S1=feats_S1, feats_S2=feats_S2, feats_S3=feats_S3, feats_S0=feats_S0,
                 dropout_p=dropout_p ) 

print(f"Number of parameters {get_n_params(model)}")

# In case we wish to transfer the learned parameters of another run
#check_file="NNs/BEST_Model_and_Optimizer_2022-04-25 17:43:51.245552_Noise_Corrector_Scratch_Network_general_MSE.pt"
#checkpoint = torch.load(save_stuff_path+f"/{check_file}")

# move model to gpu if available
model.to(device)
model.print_shapes(batch_size=2)

#model.load_state_dict(checkpoint['model'])


# Initialize the weights of the model! Default initialization might already be fine!

# we can use a MSE loss for the regression task we have in hands
#criterion = simple_denoise_criterion(0.2)
#criterion = nn.L1Loss()
criterion = nn.MSELoss()
#criterion = denoise_criterion(general_mse_over_base_mse=general_mse_over_base_mse, saturation_threshold=saturation_threshold)
# CRITERION DE SIMILITUD DE LA CORRECCIÓN AL GT PURO!!!!! IS IT MSE REALLY??

# we will choose as optimizer the 
#optimizer = torch.optim.Adagrad(model.parameters(), lr=0.1, lr_decay=0.01, weight_decay=0.3,
#                                initial_accumulator_value=0, eps=1e-10)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-9, betas=(0.9, 0.99), eps=1e-08, weight_decay=0, amsgrad=False)
#optimizer.load_state_dict(checkpoint['optimizer'])

Number of parameters 5597346
Initial shape torch.Size([2, 1, 605, 605])
Conv01 shape torch.Size([2, 5, 441, 441]) should be [2,5,441, 441]
Conv12 shape torch.Size([2, 7, 361, 361]) should be [2,7,361, 361]
Conv23 shape torch.Size([2, 10, 241, 241]) should be [2,10,241, 241]
DeConv32+mem shape torch.Size([2, 14, 361, 361]) should be [2,14,361, 361]
DeConv21+mem shape torch.Size([2, 10, 441, 441]) should be [2,10,441, 441]
DeConv10+mem shape torch.Size([2, 11, 605, 605]) should be [2,11,605, 605]
Conv00 shape torch.Size([2, 1, 605, 605]) should be [2, 1, 605, 605]


# Run the Training

In [ ]:
%%time
losses = full_training_loop(model, criterion, optimizer, training_data, training_data, 
                    epochs=total_epochs, print_loss_every_batches=10,
                            validate_every_epochs=validate_every_epochs,
                           optimizer_step_every_batches=optimizer_step_every_batches,
                           per_epoch_use_max_train_batches=per_epoch_use_max_train_batches, 
                            per_epoch_use_max_test_batches=per_epoch_use_max_test_batches,
                           image_path=save_stuff_path+ f"/Training_Loss_{datetime.now()}_{exp_name}.png",
                           save_model_every_epochs=save_model_every_epochs, 
                            model_path=save_stuff_path+f"/Model_and_Optimizer_{datetime.now()}_{exp_name}.pt",
                            best_model_path=save_stuff_path+f"/BEST_Model_and_Optimizer_{datetime.now()}_{exp_name}.pt"
                           )
# Execute the training and validation

Train Epoch: 0 [100/200000 (0%)]	Loss: 0.086247
Train Epoch: 0 [200/200000 (0%)]	Loss: 0.100946
Train Epoch: 1 [100/200000 (0%)]	Loss: 0.086335
Train Epoch: 1 [200/200000 (0%)]	Loss: 0.093885
Train Epoch: 2 [100/200000 (0%)]	Loss: 0.095950
Train Epoch: 2 [200/200000 (0%)]	Loss: 0.134671
Train Epoch: 3 [100/200000 (0%)]	Loss: 0.104133
Train Epoch: 3 [200/200000 (0%)]	Loss: 0.097746
Train Epoch: 4 [100/200000 (0%)]	Loss: 0.103756
Train Epoch: 4 [200/200000 (0%)]	Loss: 0.089054
Train Epoch: 5 [100/200000 (0%)]	Loss: 0.102452
Train Epoch: 5 [200/200000 (0%)]	Loss: 0.093325
Train Epoch: 6 [100/200000 (0%)]	Loss: 0.095746
Train Epoch: 6 [200/200000 (0%)]	Loss: 0.092683
Train Epoch: 7 [100/200000 (0%)]	Loss: 0.092688
Train Epoch: 7 [200/200000 (0%)]	Loss: 0.094182
Train Epoch: 8 [100/200000 (0%)]	Loss: 0.097721
Train Epoch: 8 [200/200000 (0%)]	Loss: 0.085148
Train Epoch: 9 [100/200000 (0%)]	Loss: 0.086427
Train Epoch: 9 [200/200000 (0%)]	Loss: 0.089147
Train Epoch: 10 [100/200000 (0%)]	Loss: 

Train Epoch: 67 [200/200000 (0%)]	Loss: 0.096784
Train Epoch: 68 [100/200000 (0%)]	Loss: 0.089204
Train Epoch: 68 [200/200000 (0%)]	Loss: 0.096443
Train Epoch: 69 [100/200000 (0%)]	Loss: 0.099570
Train Epoch: 69 [200/200000 (0%)]	Loss: 0.095706
Train Epoch: 70 [100/200000 (0%)]	Loss: 0.092572
Train Epoch: 70 [200/200000 (0%)]	Loss: 0.093104
Train Epoch: 71 [100/200000 (0%)]	Loss: 0.092494
Train Epoch: 71 [200/200000 (0%)]	Loss: 0.095541
Train Epoch: 72 [100/200000 (0%)]	Loss: 0.084607
Train Epoch: 72 [200/200000 (0%)]	Loss: 0.090374
Train Epoch: 73 [100/200000 (0%)]	Loss: 0.081405
Train Epoch: 73 [200/200000 (0%)]	Loss: 0.087533
Train Epoch: 74 [100/200000 (0%)]	Loss: 0.095096
Train Epoch: 74 [200/200000 (0%)]	Loss: 0.087632
Train Epoch: 75 [100/200000 (0%)]	Loss: 0.090964
Train Epoch: 75 [200/200000 (0%)]	Loss: 0.097153
Train Epoch: 76 [100/200000 (0%)]	Loss: 0.088688
Train Epoch: 76 [200/200000 (0%)]	Loss: 0.089450
Train Epoch: 77 [100/200000 (0%)]	Loss: 0.084718
Train Epoch: 77 [200

Train Epoch: 134 [100/200000 (0%)]	Loss: 0.106364
Train Epoch: 134 [200/200000 (0%)]	Loss: 0.090195
Train Epoch: 135 [100/200000 (0%)]	Loss: 0.078728
Train Epoch: 135 [200/200000 (0%)]	Loss: 0.082103
Train Epoch: 136 [100/200000 (0%)]	Loss: 0.094021
Train Epoch: 136 [200/200000 (0%)]	Loss: 0.098668
Train Epoch: 137 [100/200000 (0%)]	Loss: 0.082560
Train Epoch: 137 [200/200000 (0%)]	Loss: 0.090177
Train Epoch: 138 [100/200000 (0%)]	Loss: 0.088648
Train Epoch: 138 [200/200000 (0%)]	Loss: 0.105291
Train Epoch: 139 [100/200000 (0%)]	Loss: 0.082275
Train Epoch: 139 [200/200000 (0%)]	Loss: 0.101030
Train Epoch: 140 [100/200000 (0%)]	Loss: 0.091219
Train Epoch: 140 [200/200000 (0%)]	Loss: 0.090558

Validation set: Average loss: 0.0980, Average Abs Error: [2.7153983], Maximum Abs Error: [[[0.23781449 0.23781772 0.23781435 ... 0.23781694 0.23781675 0.23781653]
  [0.23782237 0.23783192 0.23783197 ... 0.23781078 0.23781766 0.2378315 ]
  [0.23781933 0.23782906 0.23782854 ... 0.23780797 0.23781726 

Train Epoch: 200 [100/200000 (0%)]	Loss: 0.086899
Train Epoch: 200 [200/200000 (0%)]	Loss: 0.080362

Validation set: Average loss: 0.0971, Average Abs Error: [2.7022347], Maximum Abs Error: [[[0.23781435 0.2378176  0.23781423 ... 0.2378168  0.23781662 0.23781638]
  [0.23782225 0.23783185 0.23783189 ... 0.23781063 0.2378175  0.23783128]
  [0.2378192  0.237829   0.23782843 ... 0.23780783 0.23781697 0.23785128]
  ...
  [0.23780674 0.23780459 0.23779887 ... 0.2378211  0.23781092 0.2378228 ]
  [0.23780678 0.23780486 0.23780444 ... 0.23782983 0.23782437 0.23782836]
  [0.2378081  0.23780747 0.23780783 ... 0.23782033 0.23781563 0.23781417]]] 

Train Epoch: 201 [100/200000 (0%)]	Loss: 0.091892
Train Epoch: 201 [200/200000 (0%)]	Loss: 0.077138
Train Epoch: 202 [100/200000 (0%)]	Loss: 0.083585
Train Epoch: 202 [200/200000 (0%)]	Loss: 0.083450
Train Epoch: 203 [100/200000 (0%)]	Loss: 0.096059
Train Epoch: 203 [200/200000 (0%)]	Loss: 0.087522
Train Epoch: 204 [100/200000 (0%)]	Loss: 0.089602
Train 

Train Epoch: 261 [100/200000 (0%)]	Loss: 0.087577
Train Epoch: 261 [200/200000 (0%)]	Loss: 0.100126
Train Epoch: 262 [100/200000 (0%)]	Loss: 0.091368
Train Epoch: 262 [200/200000 (0%)]	Loss: 0.096632
Train Epoch: 263 [100/200000 (0%)]	Loss: 0.086750
Train Epoch: 263 [200/200000 (0%)]	Loss: 0.097726
Train Epoch: 264 [100/200000 (0%)]	Loss: 0.080935
Train Epoch: 264 [200/200000 (0%)]	Loss: 0.094417
Train Epoch: 265 [100/200000 (0%)]	Loss: 0.082622
Train Epoch: 265 [200/200000 (0%)]	Loss: 0.097928
Train Epoch: 266 [100/200000 (0%)]	Loss: 0.085734
Train Epoch: 266 [200/200000 (0%)]	Loss: 0.109176
Train Epoch: 267 [100/200000 (0%)]	Loss: 0.102635
Train Epoch: 267 [200/200000 (0%)]	Loss: 0.088136
Train Epoch: 268 [100/200000 (0%)]	Loss: 0.097097
Train Epoch: 268 [200/200000 (0%)]	Loss: 0.081789
Train Epoch: 269 [100/200000 (0%)]	Loss: 0.087452
Train Epoch: 269 [200/200000 (0%)]	Loss: 0.086879
Train Epoch: 270 [100/200000 (0%)]	Loss: 0.091067
Train Epoch: 270 [200/200000 (0%)]	Loss: 0.092621


...

Train Epoch: 521 [100/200000 (0%)]	Loss: 0.095355
Train Epoch: 521 [200/200000 (0%)]	Loss: 0.101367
Train Epoch: 522 [100/200000 (0%)]	Loss: 0.096995
Train Epoch: 522 [200/200000 (0%)]	Loss: 0.087320
Train Epoch: 523 [100/200000 (0%)]	Loss: 0.095073
Train Epoch: 523 [200/200000 (0%)]	Loss: 0.084523
Train Epoch: 524 [100/200000 (0%)]	Loss: 0.085494
Train Epoch: 524 [200/200000 (0%)]	Loss: 0.087317
Train Epoch: 525 [100/200000 (0%)]	Loss: 0.075697
Train Epoch: 525 [200/200000 (0%)]	Loss: 0.079744
Train Epoch: 526 [100/200000 (0%)]	Loss: 0.076278
Train Epoch: 526 [200/200000 (0%)]	Loss: 0.072358
Train Epoch: 527 [100/200000 (0%)]	Loss: 0.077526
Train Epoch: 527 [200/200000 (0%)]	Loss: 0.082496
Train Epoch: 528 [100/200000 (0%)]	Loss: 0.085623
Train Epoch: 528 [200/200000 (0%)]	Loss: 0.098907
Train Epoch: 529 [100/200000 (0%)]	Loss: 0.087614
Train Epoch: 529 [200/200000 (0%)]	Loss: 0.089124
Train Epoch: 530 [100/200000 (0%)]	Loss: 0.090509
Train Epoch: 530 [200/200000 (0%)]	Loss: 0.088549


Train Epoch: 587 [100/200000 (0%)]	Loss: 0.074332
Train Epoch: 587 [200/200000 (0%)]	Loss: 0.080396
Train Epoch: 588 [100/200000 (0%)]	Loss: 0.087217
Train Epoch: 588 [200/200000 (0%)]	Loss: 0.083040
Train Epoch: 589 [100/200000 (0%)]	Loss: 0.078399
Train Epoch: 589 [200/200000 (0%)]	Loss: 0.084348
Train Epoch: 590 [100/200000 (0%)]	Loss: 0.090470
Train Epoch: 590 [200/200000 (0%)]	Loss: 0.084642
Train Epoch: 591 [100/200000 (0%)]	Loss: 0.081307
Train Epoch: 591 [200/200000 (0%)]	Loss: 0.080378
Train Epoch: 592 [100/200000 (0%)]	Loss: 0.085657
Train Epoch: 592 [200/200000 (0%)]	Loss: 0.080078
Train Epoch: 593 [100/200000 (0%)]	Loss: 0.078875
Train Epoch: 593 [200/200000 (0%)]	Loss: 0.087649
Train Epoch: 594 [100/200000 (0%)]	Loss: 0.085952
Train Epoch: 594 [200/200000 (0%)]	Loss: 0.081820
Train Epoch: 595 [100/200000 (0%)]	Loss: 0.081117
Train Epoch: 595 [200/200000 (0%)]	Loss: 0.084429
Train Epoch: 596 [100/200000 (0%)]	Loss: 0.078555
Train Epoch: 596 [200/200000 (0%)]	Loss: 0.086601


Train Epoch: 653 [100/200000 (0%)]	Loss: 0.093450
Train Epoch: 653 [200/200000 (0%)]	Loss: 0.077313
Train Epoch: 654 [100/200000 (0%)]	Loss: 0.078544
Train Epoch: 654 [200/200000 (0%)]	Loss: 0.079603
Train Epoch: 655 [100/200000 (0%)]	Loss: 0.080824
Train Epoch: 655 [200/200000 (0%)]	Loss: 0.074713
Train Epoch: 656 [100/200000 (0%)]	Loss: 0.071489
Train Epoch: 656 [200/200000 (0%)]	Loss: 0.079418
Train Epoch: 657 [100/200000 (0%)]	Loss: 0.091039
Train Epoch: 657 [200/200000 (0%)]	Loss: 0.078866
Train Epoch: 658 [100/200000 (0%)]	Loss: 0.084938
Train Epoch: 658 [200/200000 (0%)]	Loss: 0.078379
Train Epoch: 659 [100/200000 (0%)]	Loss: 0.081630
Train Epoch: 659 [200/200000 (0%)]	Loss: 0.082264
Train Epoch: 660 [100/200000 (0%)]	Loss: 0.090872
Train Epoch: 660 [200/200000 (0%)]	Loss: 0.089764

Validation set: Average loss: 0.0902, Average Abs Error: [2.5993214], Maximum Abs Error: [[[0.23781337 0.23781669 0.23781325 ... 0.2378157  0.23781553 0.23781529]
  [0.23782136 0.23783123 0.23783132 

Train Epoch: 719 [100/200000 (0%)]	Loss: 0.083938
Train Epoch: 719 [200/200000 (0%)]	Loss: 0.089136
Train Epoch: 720 [100/200000 (0%)]	Loss: 0.087298
Train Epoch: 720 [200/200000 (0%)]	Loss: 0.074180

Validation set: Average loss: 0.0891, Average Abs Error: [2.5806243], Maximum Abs Error: [[[0.23781325 0.23781656 0.23781313 ... 0.23781554 0.23781538 0.23781516]
  [0.23782124 0.23783118 0.23783123 ... 0.23780924 0.2378161  0.2378293 ]
  [0.23781809 0.23782857 0.2378272  ... 0.2378065  0.23781526 0.2378489 ]
  ...
  [0.23780553 0.23780337 0.2377976  ... 0.23781843 0.23780975 0.23782164]
  [0.23780558 0.23780365 0.23780322 ... 0.23782796 0.23782271 0.237827  ]
  [0.2378069  0.23780622 0.23780656 ... 0.23781839 0.2378145  0.237813  ]]] 

Train Epoch: 721 [100/200000 (0%)]	Loss: 0.074096
Train Epoch: 721 [200/200000 (0%)]	Loss: 0.083994
Train Epoch: 722 [100/200000 (0%)]	Loss: 0.077354
Train Epoch: 722 [200/200000 (0%)]	Loss: 0.070610
Train Epoch: 723 [100/200000 (0%)]	Loss: 0.083037
Train 

Train Epoch: 781 [100/200000 (0%)]	Loss: 0.081354
Train Epoch: 781 [200/200000 (0%)]	Loss: 0.087376
Train Epoch: 782 [100/200000 (0%)]	Loss: 0.087461
Train Epoch: 782 [200/200000 (0%)]	Loss: 0.086164
Train Epoch: 783 [100/200000 (0%)]	Loss: 0.076428
Train Epoch: 783 [200/200000 (0%)]	Loss: 0.086545
Train Epoch: 784 [100/200000 (0%)]	Loss: 0.069847
Train Epoch: 784 [200/200000 (0%)]	Loss: 0.079250
Train Epoch: 785 [100/200000 (0%)]	Loss: 0.073575
Train Epoch: 785 [200/200000 (0%)]	Loss: 0.080813
Train Epoch: 786 [100/200000 (0%)]	Loss: 0.086282
Train Epoch: 786 [200/200000 (0%)]	Loss: 0.073853
Train Epoch: 787 [100/200000 (0%)]	Loss: 0.067690
Train Epoch: 787 [200/200000 (0%)]	Loss: 0.083187
Train Epoch: 788 [100/200000 (0%)]	Loss: 0.083426
Train Epoch: 788 [200/200000 (0%)]	Loss: 0.076564
Train Epoch: 789 [100/200000 (0%)]	Loss: 0.074491
Train Epoch: 789 [200/200000 (0%)]	Loss: 0.078411
Train Epoch: 790 [100/200000 (0%)]	Loss: 0.085160
Train Epoch: 790 [200/200000 (0%)]	Loss: 0.088764


# Save the resulting model weights

In [ ]:
torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            }, save_stuff_path+f"FINAL_Model_and_Optimizer_{datetime.now()}_{exp_name}.pt")

# Final Validation

In [ ]:
print("\n\n\nFINAL VALIDATION! ####################################################\n\n")
print("Train Set")
#validate_epoch(nn.MSELoss(), model, sampler, dataset, per_epoch_use_max_train_batches)
print("Test Set")
validate_epoch(nn.MSELoss(), model, test_dataloader, per_epoch_use_max_test_batches)

In [ ]:
hide_toggle(True)

In [ ]:
def compute_intensity_gravity_centers(images):
    """
        Expects input image to be an array of dimensions [N_imgs, h, w].
        It will return an array of gravity centers [N_imgs, 2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to array indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = torch.sum(images, dim=1) # weights for x [N_images, raw_width]
    intensity_in_h = torch.sum(images, dim=2) # weights for y [N_images, raw_height]
    total_intensity = intensity_in_h.sum(dim=1) # [N_images]

    # Compute mass center for intensity
    # [N_images, 2] (h_center,w_center)
    return torch.nan_to_num( torch.stack(
        (torch.matmul(intensity_in_h.float(), torch.arange(images.shape[1], 
                                    dtype=torch.float32, device=device))/total_intensity,
         torch.matmul(intensity_in_w.float(), torch.arange(images.shape[2], 
                                    dtype=torch.float32, device=device))/total_intensity),
        dim=1
        ), nan=0.0, posinf=None, neginf=None)

def compute_raw_to_centered_iX(images, X=302):

        g_raw = compute_intensity_gravity_centers(images) # [ N_images, 2]

        # crop the iamges with size (X+1+X)^2 leaving the gravity center in
        # the central pixel of the image. In case the image is not big enough for the cropping,
        # a 0 padding will be made.
        centered_images = torch.zeros( ( images.shape[0], 2*X+1, 2*X+1),  dtype = images.dtype, 
                                      device=device)

        # we round the gravity centers to the nearest pixel indices
        g_index_raw = torch.round(g_raw).int() #[ N_images, 2]

        # obtain the slicing indices around the center of gravity
        # TODO -> make all this with a single array operation by stacking the lower and upper in
        # a new axis!!
        # [ N_images, 2 (h,w)]
        unclipped_lower = g_index_raw-X
        unclipped_upper = g_index_raw+X+1

        # unclipped could get out of bounds for the indices, so we clip them
        lower_bound = torch.clip( unclipped_lower.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        upper_bound = torch.clip( unclipped_upper.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        # we use the difference between the clipped and unclipped to get the necessary padding
        # such that the center of gravity is left still in the center of the image
        padding_lower = lower_bound-unclipped_lower
        padding_upper = upper_bound-unclipped_upper

        # crop the image
        for im in range(g_raw.shape[0]):
            centered_images[im, padding_lower[ im, 0]:padding_upper[ im, 0] or None,
                                        padding_lower[ im, 1]:padding_upper[ im, 1] or None] = \
                      images[im, lower_bound[ im, 0]:upper_bound[ im, 0],
                                          lower_bound[ im, 1]:upper_bound[ im, 1]]

        return centered_images

In [ ]:
import os
import cv2
plot3d_resolution=0.3

path = "/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/UMAP_Regressor/TEST_IMAGES/"
image_names = os.listdir(f"{path}")

predicted_corrections={}
problem_images = {}
os.makedirs(f"{save_stuff_path}/Test_Corrections/", exist_ok=True)

for im_n in image_names:
    model.eval()
    im = cv2.imread(path+im_n, cv2.IMREAD_ANYDEPTH)
    im_type = im.dtype
    max_int = 2**8-1 if im_type==np.uint8 else 2**16-1 if im_type==np.uint16 else 2**32-1 if im_type==np.uint32 else 2**53-1 if im_type==np.uint64 else None
    im = compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))
    corr_im = model(im)[0] #[1, 2X+1, 2X+1]
    corr_im = max_int*(torch.abs(corr_im/corr_im.amax(dim=(1,2), keepdim=True)[0].unsqueeze(1)))
    corr_im = np.asarray(corr_im.detach().to('cpu').squeeze(0)).astype(im_type)
    predicted_corrections[im_n] = corr_im
    im = np.asarray(im.detach().to('cpu').squeeze(0)).astype(im_type)
    problem_images[im_n] = im
    
    # On the one hand we want to plot the resulting corrections on the test set
    fig = plt.figure(figsize=(2*4.5, 2*4.5))
    axes=fig.subplots(2,2)

    cm=axes[0, 0].imshow(im, cmap='viridis')
    cm2 = axes[0,1].imshow(corr_im, cmap='viridis')
    axes[0,0].grid(True)
    axes[0,1].grid(True)

    axes[1,0].set_visible(False)
    axes[1,1].set_visible(False)
    ax = fig.add_subplot(223, projection='3d')
    Xs,Ys = np.meshgrid(np.arange(im.shape[0]),np.arange(im.shape[1]))
    fig.suptitle(f"Intesity Profiles for Image\n{im_n}")
    cbax=fig.add_axes([0.54,0.05,0.4,0.01])
    fig.colorbar(cm, ax=axes[0,0], cax=cbax, orientation='horizontal')
    theta=25
    phi=30
    ax.plot_surface(Xs, Ys, im.T, rcount=int(im.shape[1]*plot3d_resolution), 
                    ccount=int(im.shape[0]*plot3d_resolution), cmap='viridis') # rstride=1, cstride=1, linewidth=0
    ax.set_xlabel('Y')
    #ax.set_xlim(-8, 8)
    ax.set_ylabel('X')
    #ax.set_ylim(-10, 8)
    ax.set_zlabel('Intensity')
    ax.set_zlim(-0.078*np.max(im), np.max(im))
    ax.set_title("Image intensity 3D plot")
    ax.view_init(10, theta)
    #ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.3, 1.3, 1.3, 1]))

    ax = fig.add_subplot(224, projection='3d')
    Xs,Ys = np.meshgrid(np.arange(corr_im.shape[0]),np.arange(corr_im.shape[1]))
    fig.suptitle(f"Intesity Profiles for Image\n{im_n}")
    theta=25
    phi=30
    ax.plot_surface(Xs, Ys, corr_im.T, rcount=int(corr_im.shape[0]*plot3d_resolution), ccount=int(corr_im.shape[1]*plot3d_resolution), cmap='viridis') # rstride=1, cstride=1, linewidth=0
    ax.set_xlabel('Y')
    #ax.set_xlim(-8, 8)
    ax.set_ylabel('X')
    #ax.set_ylim(-10, 8)
    ax.set_zlabel('Intensity')
    ax.set_zlim(-0.078*np.max(corr_im), np.max(corr_im))
    ax.set_title("Image intensity 3D plot")
    ax.view_init(10, theta)
    #ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.3, 1.3, 1.3, 1]))
    fig.savefig(f"{save_stuff_path}/Test_Corrections/{im_n}")
    print(f"{im_n} processed by the denoiser and image saved")
    
    


In [ ]:

# We now save them with the correct directory structure such that we can use the Todor Algorithms on them
# This will work as a benchmark of how well the correction was made
for i in ["Problem", "Reference"]:
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/17_18/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/18_19/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/28_29/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/43_44/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/70_71/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/con_los_dos/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/non_noisy_5_6/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/non_noisy_72_73/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/ortog/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/ref_vs_ref/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/{i}", exist_ok=True)
    os.makedirs( f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/{i}", exist_ok=True)
    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/17_18/Problem/Corr_17.png", predicted_corrections['IM_53017_phiCR_0.659442126750946.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/17_18/Reference/Corr_18.png", predicted_corrections['IM_53018_phiCR_-2.2813968658447266.png'])    

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/18_19/Problem/Corr_18.png", predicted_corrections['IM_53018_phiCR_-2.2813968658447266.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/18_19/Reference/Corr_19.png", predicted_corrections['IM_53019_phiCR_-2.679948091506958.png'])    

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/28_29/Problem/Corr_28.png", predicted_corrections['IM_52928_phiCR_0.6789670586585999.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/28_29/Reference/Corr_29.png", predicted_corrections['IM_52929_phiCR_0.9714600443840027.png'])    

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/43_44/Problem/Corr_43.png", predicted_corrections['IM_43_phiCR_-1.57120680809021.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/43_44/Reference/Corr_44.png", predicted_corrections['IM_44_phiCR_2.6544740200042725.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/70_71/Problem/Corr_70.png", predicted_corrections['IM_40870_phiCR_-0.6731816530227661.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/70_71/Reference/Corr_71.png", predicted_corrections['IM_40871_phiCR_-2.4470927715301514.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/con_los_dos/Problem/Corr_con_los_dos.png", predicted_corrections['con_los_dos.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/con_los_dos/Reference/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/con_los_dos/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  


cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/Problem/Corr_sin_el_negativo.png", predicted_corrections['sin_el_negativo.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/Reference/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_negativo/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  


cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/Problem/Corr_sin_el_positivo.png", predicted_corrections['sin_el_positivo.png'])    
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/Reference/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/sin_el_positivo/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ref_vs_ref/Problem/Corr_ref1.png", predicted_corrections['sin_los_dos_solo_tubo.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ref_vs_ref/Reference/Corr_ref2.png", predicted_corrections['antes_de_la_estandar.png'])  


cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ortog/Problem/Corr_90.png", predicted_corrections['90__100.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/ortog/Reference/Corr_Ref_90.png", predicted_corrections['Reference__100.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_5_6/Problem/Corr_5.png", predicted_corrections['IM_5_phiCR_-2.6049387454986572.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_5_6/Reference/Corr_6.png", predicted_corrections['IM_6_phiCR_-1.7562638521194458.png'])  

cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_72_73/Problem/Corr_72.png", predicted_corrections['IM_72_phiCR_-2.946422576904297.png'])  
cv2.imwrite(f"{save_stuff_path}/Todor_Benchmark/non_noisy_72_73/Reference/Corr_73.png", predicted_corrections['IM_73_phiCR_1.33404541015625.png'])  



In [ ]:
print("\n\nProceeding to apply Todor Algorithms...\n")
os.system(f"python ../../ANALYSIS_SCRIPTS/CODE_Get_Angle_Live.py {save_stuff_path+'/Todor_Benchmark/'} {0.05} False")
print("Done!")

In [ ]:
raise ValueError

# Charge models and do inference

In [ ]:
checkpoint = torch.load(f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/Simple_Encoder/Noisy_Model_and_Optimizer_2022-01-24 19:57:31.886991.pt")

model = Simple_Encoder( X=X, feats_1=feats_1, feats_2=feats_2, feats_3=feats_3, feats_4=feats_4,
                 prop1=prop1, prop2=prop2, prop3=prop3, av_pool1_div=av_pool1_div, conv4_feat_size=conv4_feat_size, av_pool2_div=av_pool2_div, 
                 out_fc_1=out_fc_1,
                 dropout_p1=dropout_p1, dropout_p2=dropout_p2 ) 

model.to(device)
model.load_state_dict(checkpoint['model'])

In [ ]:
def compute_intensity_gravity_centers(images):
    """
        Expects input image to be an array of dimensions [N_imgs, h, w].
        It will return an array of gravity centers [N_imgs, 2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to array indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = torch.sum(images, dim=1) # weights for x [N_images, raw_width]
    intensity_in_h = torch.sum(images, dim=2) # weights for y [N_images, raw_height]
    total_intensity = intensity_in_h.sum(dim=1) # [N_images]

    # Compute mass center for intensity
    # [N_images, 2] (h_center,w_center)
    return torch.nan_to_num( torch.stack(
        (torch.matmul(intensity_in_h.float(), torch.arange(images.shape[1], 
                                    dtype=torch.float32, device=device))/total_intensity,
         torch.matmul(intensity_in_w.float(), torch.arange(images.shape[2], 
                                    dtype=torch.float32, device=device))/total_intensity),
        dim=1
        ), nan=0.0, posinf=None, neginf=None)

def compute_raw_to_centered_iX(images, X=302):

        g_raw = compute_intensity_gravity_centers(images) # [ N_images, 2]

        # crop the iamges with size (X+1+X)^2 leaving the gravity center in
        # the central pixel of the image. In case the image is not big enough for the cropping,
        # a 0 padding will be made.
        centered_images = torch.zeros( ( images.shape[0], 2*X+1, 2*X+1),  dtype = images.dtype, 
                                      device=device)

        # we round the gravity centers to the nearest pixel indices
        g_index_raw = torch.round(g_raw).int() #[ N_images, 2]

        # obtain the slicing indices around the center of gravity
        # TODO -> make all this with a single array operation by stacking the lower and upper in
        # a new axis!!
        # [ N_images, 2 (h,w)]
        unclipped_lower = g_index_raw-X
        unclipped_upper = g_index_raw+X+1

        # unclipped could get out of bounds for the indices, so we clip them
        lower_bound = torch.clip( unclipped_lower.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        upper_bound = torch.clip( unclipped_upper.float(), min=torch.Tensor([[0,0]]).to(device),
                                 max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
        # we use the difference between the clipped and unclipped to get the necessary padding
        # such that the center of gravity is left still in the center of the image
        padding_lower = lower_bound-unclipped_lower
        padding_upper = upper_bound-unclipped_upper

        # crop the image
        for im in range(g_raw.shape[0]):
            centered_images[im, padding_lower[ im, 0]:padding_upper[ im, 0] or None,
                                        padding_lower[ im, 1]:padding_upper[ im, 1] or None] = \
                      images[im, lower_bound[ im, 0]:upper_bound[ im, 0],
                                          lower_bound[ im, 1]:upper_bound[ im, 1]]

        return centered_images

In [ ]:
# Create and display a FileChooser widget
from ipyfilechooser import FileChooser
path="/home/oiangu/Desktop/Conical_Refraction_Polarimeter"
fc = FileChooser(path+'/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/')
display(fc)

### Choose a single experimental image to predict

In [ ]:
import cv2
image_full_path=fc.selected
#image_full_path="/home/oiangu/Desktop/Conical_Refraction_Polarimeter/Experimental_Stuff/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_el_positivo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)
if im is None:
    print(f" Unable to import image {image_full_path}")
    raise ValueError
# Center in gravicenter, generating iX
im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

### Plot its Profiles

In [ ]:
plot3d_resolution=0.7

%matplotlib notebook

prof_x=np.sum(im, axis=0)
prof_y=np.sum(im, axis=1)
fig = plt.figure(figsize=(2*4.5, 2*4.5))
axes=fig.subplots(2,2)

cm=axes[0, 0].imshow(im, cmap='viridis')
axes[0,0].grid(True)
axes[0,1].scatter(prof_y, np.arange(len(prof_y)), s=1, label=f'Intensity profile in y')
axes[0,1].set_ylim((0,len(prof_y)))
axes[0,1].invert_yaxis()
axes[1,0].scatter(np.arange(len(prof_x)), prof_x, s=1, label=f'Intensity profile in y')
axes[1,0].set_xlim((0,len(prof_x)))
axes[1,0].invert_yaxis()
axes[0,0].set_xlabel("x (pixels)")
#axes[0,0].set_ylabel("y (pixels)")
axes[0,1].set_xlabel("Cummulative Intensity")
axes[0,1].set_ylabel("y (pixels)")
axes[1,0].set_ylabel("Cummulative Intensity")
axes[1,0].set_xlabel("x (pixels)")
axes[1,0].grid(True)
axes[0,1].grid(True)
axes[1,1].set_visible(False)
ax = fig.add_subplot(224, projection='3d')
Xs,Ys = np.meshgrid(np.arange(len(prof_y)),np.arange(len(prof_x)))
fig.suptitle(f"Intesity Profiles for Image\n{image_full_path.split('/')[-1]}")
files_for_gif=[]
cbax=fig.add_axes([0.54,0.05,0.4,0.01])
fig.colorbar(cm, ax=axes[0,0], cax=cbax, orientation='horizontal')
theta=25
phi=30
ax.plot_surface(Xs, Ys, im.T, rcount=int(len(prof_y)*plot3d_resolution), ccount=int(len(prof_x)*plot3d_resolution), cmap='viridis') # rstride=1, cstride=1, linewidth=0
#cset = ax.contourf(X, Y, im, 2, zdir='z', offset=-20, cmap='viridis', alpha=0.5)
#cset = ax.contourf(X, Y, im, 1, zdir='x', offset=-8, cmap='viridis')
#cset = ax.contourf(X, Y, im, 1, zdir='y', offset=0, cmap='viridis')
ax.set_xlabel('Y')
#ax.set_xlim(-8, 8)
ax.set_ylabel('X')
#ax.set_ylim(-10, 8)
ax.set_zlabel('Intensity')
ax.set_zlim(-0.078*np.max(im), np.max(im))
ax.set_title("Image intensity 3D plot")
ax.view_init(10, theta)
#ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.3, 1.3, 1.3, 1]))
plt.show()

### Get NN predictions for $R_0, w_0, \phi_{CR}, Z$

In [ ]:
print("Custom")
model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
print(f"Predicted phi_CR {predictions[0]} rad {predictions[0]*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {predictions[0]/2} rad {predictions[0]*180/np.pi/2} deg")

In [ ]:
import cv2
print("Referencia sin nada\n")
%matplotlib inline
image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_los_dos_solo_tubo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ref=predictions[0].item()
print(f"Predicted phi_CR {ref} rad {ref*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ref/2} rad {ref*180/np.pi/2} deg")

In [ ]:
print("Referencia sin nada\n")
%matplotlib inline
image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/antes_de_la_estandar.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ref2=predictions[0].item()
print(f"Predicted phi_CR {ref2} rad {ref2*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ref2/2} rad {ref2*180/np.pi/2} deg")

In [ ]:
print("Sin el negativo\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_el_negativo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
pos=predictions[0].item()
print(f"Predicted phi_CR {pos} rad {pos*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {pos/2} rad {pos*180/np.pi/2} deg")

In [ ]:
print("Sin el positivo\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/sin_el_positivo.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
neg = predictions[0].item()
print(f"Predicted phi_CR {neg} rad {neg*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {neg/2} rad {neg*180/np.pi/2} deg")

In [ ]:
print("Con ambos\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day2/laser_gaussian_thesis/All_Taken_Photos/con_los_dos.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
both=predictions[0].item()
print(f"Predicted phi_CR {both} rad {both*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {both/2} rad {both*180/np.pi/2} deg")


In [ ]:
print("Ref Ort\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day3/Reference/Reference__100.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ref_ort=predictions[0].item()
print(f"Predicted phi_CR {ref_ort} rad {ref_ort*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ref_ort/2} rad {ref_ort*180/np.pi/2} deg")


In [ ]:
print("Ref Ort\n")
%matplotlib inline

image_full_path = "/home/oiangu/Desktop/Conical_Refraction_Polarimeter/LAB/EXPERIMENTAL/Fotos_Turpin/Day3/Problem/90__100.png"
im = cv2.imread(image_full_path, cv2.IMREAD_ANYDEPTH)

im = np.asarray((compute_raw_to_centered_iX(torch.from_numpy(im).unsqueeze(0).to(device))).to('cpu').squeeze(0))
plt.imshow(im)
plt.show()

model.eval()
predictions = model(torch.from_numpy(im).to(device).unsqueeze(0))[0]
ort=predictions[0].item()
print(f"Predicted phi_CR {ref_ort} rad {ort*180/np.pi} deg")
print(f"\n\nPredicted Polarization plane -relative to the image plane w axis- is {ort/2} rad {ort*180/np.pi/2} deg")


In [ ]:
print(f"Positivo-Ref deberian ser {13.85} deg son {(pos-ref)*180/np.pi/2} deg")
print(f"Negativo-Ref deberian ser {9.45} deg son {(neg-ref)*180/np.pi/2} deg")
print(f"Ambos-Ref deberian ser {4.4} deg son {(both-ref)*180/np.pi/2} deg\n")

print(f"Positivo-Ref2 deberian ser {13.85} deg son {(pos-ref2)*180/np.pi/2} deg")
print(f"Negativo-Ref2 deberian ser {9.45} deg son {(neg-ref2)*180/np.pi/2} deg")
print(f"Ambos-Ref2 deberian ser {4.4} deg son {(both-ref2)*180/np.pi/2} deg\n")

print(f"Ref2-Ref deberian ser {0} deg son {(ref2-ref)*180/np.pi/2} deg\n")

print(f"El de noventa deberian ser {90} deg son {(ref_ort-ort)*180/np.pi/2} deg")

### Get the non-black-box algorithm estimate for $\phi_{CR}$

In [ ]:
import os
os.chdir(f"../../..")
import sys
from SOURCE.CLASS_CODE_GPU_Classes import *
from SOURCE.CLASS_CODE_Image_Manager import *
from SOURCE.CLASS_CODE_Polarization_Obtention_Algorithms import Rotation_Algorithm, Mirror_Flip_Algorithm, Gradient_Algorithm
import numpy as np
import json
import cv2
import pandas as pd
import matplotlib.pyplot as plt

image=im.copy()
saturation=0.9
pol_or_CR="pol" 
deg_or_rad="deg" # for the final output
image_depth=8 # or 16 bit per pixel
image_shortest_side=540
randomization_seed=666
recenter_average_image=False


# 5. POLARIZATION RELATIVE ANGLES ###################################
# Mirror with affine interpolation & Rotation Algorithms will be employed
# Each using both Fibonacci and Quadratic Fit Search
# Also a gradient algorithm
theta_min_Rot=-np.pi
theta_max_Rot=np.pi
rad_min_Grav=3
rad_max_Grav=image_shortest_side
theta_min_Mir=0
theta_max_Mir=np.pi
initial_guess_delta_rad=0.1
initial_guess_delta_pix=10
use_exact_gravicenter=True
precision_quadratic=1e-10
max_it_quadratic=100
cost_tolerance_quadratic=1e-14
precision_fibonacci=1e-10
max_points_fibonacci=100
cost_tolerance_fibonacci=1e-14


##################################################################
##################################################################
im_type=np.uint16 if image_depth==16 else np.uint8
max_intensity=65535 if image_depth==16 else 255
np.random.seed(randomization_seed)
polCR=1 if pol_or_CR=='CR' else 0.5

# 6. POLARIZATION RELATIVE ANGLES ###################################
# Mirror with affine interpolation & Rotation Algorithms will be employed
# Each using both Fibonacci and Quadratic Fit Search
# Results will be gathered in a table and outputed as an excel csv
# Mock Image Loader
# Computar el angulo de cada uno en un dataframe donde una de las entradas sea results y haya un result per fibo qfs y per rotation y mirror affine. Y luego procesar en un 7º paso estos angulos para obtener los angulos relativos etc y perhaps hacer tablucha con ground truth menos el resulting delta angle medido por el algoritmo
image_loader = Image_Manager(mode=X, interpolation_flag=None)
# Define the ROTATION ALGORITHM
rotation_algorithm = Rotation_Algorithm(image_loader,
    theta_min_Rot, theta_max_Rot, None,
    initial_guess_delta_rad, use_exact_gravicenter, initialize_it=False)

# Define the Affine Mirror algorithm
mirror_algorithm = Mirror_Flip_Algorithm(image_loader,
    theta_min_Mir, theta_max_Mir, None,
    initial_guess_delta_rad, method="aff", left_vs_right=True, use_exact_gravicenter=use_exact_gravicenter, initialize_it=False)

# Define the Gradient algorithm
gradient_algorithm = Gradient_Algorithm(image_loader,
        rad_min_Grav, rad_max_Grav,
        initial_guess_delta_pix,
        use_exact_gravicenter)

# A dictionary to gather all the resulting angles for each image

individual_image_results = { 'polarization_method':[], 'optimization_1d':[], 'found_phiCR':[], 'predicted_opt_precision':[] }

def to_result_dict(result_dict, alg, alg_name, opt_name, im_names):
    for key, name in zip(alg.times.keys(), im_names):
        result_dict['polarization_method'].append(alg_name)
        result_dict['optimization_1d'].append(opt_name)
        result_dict['found_phiCR'].append(alg.angles[key])
        result_dict['predicted_opt_precision'].append(alg.precisions[key])
image_container=np.zeros( (1, 2*X+1, 2*X+1), dtype=np.float64)
image_names=[]
# charge the image
image_container[0]=image.astype(np.float64)
image_names.append(f"{fc.selected_filename}")

# charge the image loader:
image_loader.import_converted_images_as_array(image_container, image_names)
# Execute the Rotation and Mirror Algorithms:
# ROTATION ######
interpolation_flag=None
# the interpolation algorithm used in case we disbale its usage for the iX image obtention will be the Lanczos one
rotation_algorithm.interpolation_flag=interpolation_flag if interpolation_flag is not None else cv2.INTER_CUBIC
rotation_algorithm.reInitialize(image_loader)
rotation_algorithm.quadratic_fit_search(precision_quadratic, max_it_quadratic, cost_tolerance_quadratic)
to_result_dict( individual_image_results, rotation_algorithm, "Rotation", "Quadratic", image_names)
rotation_algorithm.reInitialize(image_loader)
rotation_algorithm.fibonacci_ratio_search(precision_fibonacci, max_points_fibonacci, cost_tolerance_fibonacci)
to_result_dict( individual_image_results, rotation_algorithm, "Rotation", "Fibonacci", image_names)

# MIRROR #######
mirror_algorithm.interpolation_flag=interpolation_flag if interpolation_flag is not None else cv2.INTER_CUBIC
mirror_algorithm.reInitialize(image_loader)
mirror_algorithm.quadratic_fit_search(precision_quadratic, max_it_quadratic, cost_tolerance_quadratic)
to_result_dict( individual_image_results, rotation_algorithm, "Mirror", "Quadratic", image_names)
mirror_algorithm.reInitialize(image_loader)
mirror_algorithm.fibonacci_ratio_search(precision_fibonacci, max_points_fibonacci, cost_tolerance_fibonacci)
to_result_dict( individual_image_results, rotation_algorithm, "Mirror", "Fibonacci", image_names)

# GRADIENT #######
def compute_intensity_gravity_center(image):
    """
        Expects input image to be an array of dimensions [h, w].
        It will return an array of gravity centers [2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to numpy indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = np.sum(image, axis=0) # weights for x [raw_width]
    intensity_in_h = np.sum(image, axis=1) # weights for y [raw_height]
    total_intensity = intensity_in_h.sum()

    # Compute mass center for intensity
    # [2] (h_center,w_center)
    return np.nan_to_num( np.stack(
        (np.dot(intensity_in_h, np.arange(image.shape[0]))/total_intensity,
         np.dot(intensity_in_w, np.arange(image.shape[1]))/total_intensity)
        ) )

optimal_masked_gravs={}
optimal_radii={}
grav=compute_intensity_gravity_center(image)

gradient_algorithm.interpolation_flag=interpolation_flag if interpolation_flag is not None else cv2.INTER_CUBIC
gradient_algorithm.reInitialize(image_loader)
gradient_algorithm.quadratic_fit_search(precision_quadratic, max_it_quadratic, cost_tolerance_quadratic)
to_result_dict( individual_image_results, gradient_algorithm, "Gradient", "Quadratic", image_names)
#optimal_masked_gravs['quad'] = gradient_algorithm.masked_gravs[f"Quadratic_Search_{fc.selected_filename}"]
#optimal_radii['quad'] = gradient_algorithm.optimals[f"Quadratic_Search_{fc.selected_filename}"]

gradient_algorithm.reInitialize(image_loader)
gradient_algorithm.fibonacci_ratio_search(precision_fibonacci, max_points_fibonacci, cost_tolerance_fibonacci)
to_result_dict( individual_image_results, gradient_algorithm, "Gradient", "Fibonacci", image_names)

#optimal_masked_gravs['fibo'] = gradient_algorithm.masked_gravs[f"Fibonacci_Search_{fc.selected_filename}"]
#optimal_radii['fibo'] = gradient_algorithm.optimals[f"Fibonacci_Search_{fc.selected_filename}"]

#masked_grav=(optimal_masked_gravs['quad']+optimal_masked_gravs['fibo'])/2.0
#optimal_radi = (optimal_radii['quad']+optimal_radii['fibo'])/2
#print(f"\n\nOptimal masked gravs: {optimal_masked_gravs}\nOptimal radii: {optimal_radii}\n\n\n")
print(pd.DataFrame.from_dict(individual_image_results))

# 7. PROCESS FINAL RESULTS ##########################################
def angle_to_pi_pi( angle): # convert any angle to range ()-pi,pi]
    angle= angle%(2*np.pi) # take it to [-2pi, 2pi]
    return angle-np.sign(angle)*2*np.pi if abs(angle)>np.pi else angle    

average_found_phiCR=np.mean([angle_to_pi_pi(phi) for i,phi in enumerate(individual_image_results['found_phiCR']) if individual_image_results['polarization_method'][i]!='Gradient'])
print("Average found phiCR:", average_found_phiCR)
#print(f"\n\nPredicted slope for main axis: by Gradient {(masked_grav[0]-grav[0])/(masked_grav[1]-grav[1])} and by the others averaged {np.tan(-average_found_phiCR)}")

In [ ]:
'''
def forward(self, x): # [batch_size, 2X+1, 2X+1] or [batch_size, 1, 2X+1, 2X+1]
    x = x.view(x.shape[0], 1, x.shape[-2], x.shape[-1]) # [batch_size, 1, 2X+1, 2X+1]
    X=302
    feats_1=15
    feats_2=20
    feats_3=20
    feats_4=20
    prop1=3
    prop2=2
    prop3=1
    av_pool1_div=4
    conv4_feat_size=15
    av_pool2_div=10
    out_fc_1=10 
    print(x.shape, 2*X+1)

    x = self.relu( self.conv1(x) ) # [batch_size, feats_1, prop1*(2X+1)/5, prop1*(2X+1)/5]
    print("conv1",x.shape, prop1*(2*X+1)/5)


    x = self.batchNorm2( self.relu( self.conv2(self.dropout1(x)) )) # [batch_size, feats_2, prop2*(2X+1)/5, prop2*(2X+1)/5]
    print("conv2",x.shape,  prop2*(2*X+1)/5)


    x = self.relu( self.conv3(self.dropout2(x)) ) # [batch_size, feats_3, prop3*(2X+1)/5, prop3*(2X+1)/5]
    print("conv3",x.shape,  prop3*(2*X+1)/5)


    x = self.avPool1(x) # [batch_size, feats_3, prop3*(2X+1)/5)/av_pool1_div, prop3*(2X+1)/5)/av_pool1_div]
    print("av_pool1",x.shape, int((prop3*(2*X+1)/5)/av_pool1_div))


    x = self.batchNorm4(self.conv4(self.dropout2(x))) # [batch_size, feats_4, conv4_feat_size, conv4_feat_size]
    print("conv4+batchn",x.shape, conv4_feat_size)


    x = self.relu( self.avPool2(x) ) # [batch_size, feats_4, conv4_feat_size/av_pool2_div, conv4_feat_size/av_pool2_div]
    print("av_pool2",x.shape, int(conv4_feat_size/av_pool2_div)+1)


    x = x.view(x.shape[0], self.in_fc) #[batch_size, feats_4*int(conv4_feat_size/av_pool2_div)**2]
    print("view_change",x.shape, feats_4*int(conv4_feat_size/av_pool2_div+1)**2)


    x = self.fc2( self.relu( self.fc1(x) ) ) #[batch_size, 4]
    print(x.shape, 4)

        return x
a = Simple_Encoder().to(device)
a(torch.ones(2,1, 605,605).to(device))
del a
torch.cuda.empty_cache()
'''

In [ ]:
def train_crazy_epoch(epoch, criterion, model, optimizer, datas, targets, batch_number, batch_size,
                      print_loss_every_batches=20,
                    optimizer_step_every_batches=1):
    
    total_loss = 0.0

    model.train()

    optimizer.zero_grad()
    t2 = time()
    for k in range(batch_number):        
        
        prediction = model(datas[k*batch_size:(k+1)*batch_size]) # data is [batch_size, 1, 2X+1, 2X+1]
        loss = criterion(prediction, targets[k*batch_size:(k+1)*batch_size])
        loss.backward()
        
        if k % optimizer_step_every_batches==optimizer_step_every_batches-1:
            optimizer.step()
            optimizer.zero_grad()
        # print loss every N batches
        if k % print_loss_every_batches == print_loss_every_batches-1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (k+1) * batch_size, len(datas),
                100*(k+1)*batch_size / len(datas), loss.item()))

        #total_loss += loss.item()  #.item() is very important here
        # Why?-> In order to avoid having total_loss as a tensor in the gpu
        t1= time()
        print(f"Iteration time{t1-t2}")
        t2 = time()

    return total_loss / len(datas)

In [ ]:
def full_crazy_training_loop(model, criterion, optimizer_generator, train_loader,
                             batch_number, batch_size, epochs=10,
                       print_loss_every_batches=20, optimizer_step_every_batches=1,
                            meta_epoch_number=1):
    %matplotlib inline
    for meta_epoch in range(meta_epoch_number):
        for meta_batch_id, (datas, targets) in enumerate(train_loader):        
            datas, targets = datas.to(device), targets.to(device) # pero muuh gordos
            losses = {"train": []}
            optimizer = optimizer_generator(model)
            for epoch in range(epochs): # que overfitee el muuh gordo este
                train_loss = train_crazy_epoch(epoch, criterion, model, optimizer, datas,
                                         targets, batch_number, batch_size,
                                          print_loss_every_batches=20,
                                            optimizer_step_every_batches=1)

                display_IPython.clear_output(wait=True)
                losses["train"].append(train_loss)
                plt.plot(losses["train"], label=f"log training loss- MetaBatch {meta_batch_id/len(train_loader)*100}%")
                plt.yscale('log')
                plt.legend()
                plt.pause(0.001)
                plt.show()   
    return losses

In [ ]:
meta_epoch_number = 1
meta_batch_size = 100
batch_size = 10
batch_number = int(meta_batch_size/batch_size)
assert(meta_batch_size%batch_size==0)

crazy_loader = DataLoader(training_data, batch_size=meta_batch_size, shuffle=True, num_workers=worker_num,
                              pin_memory=True, drop_last=False, persistent_workers=False)

def adam_generator(model):
    return torch.optim.Adam(model.parameters(), lr=0.05, betas=(0.99, 0.9999), eps=1e-08, weight_decay=0, amsgrad=False)

full_crazy_training_loop(model, criterion, 
                         adam_generator, 
                         crazy_loader,
                             batch_number, batch_size, epochs=10,
                       print_loss_every_batches=10, optimizer_step_every_batches=2, 
                         meta_epoch_number=meta_epoch_number)

In [ ]:
t1=time()
for datas, targets in train_dataloader:
    datas, targets = datas.to(device), targets.to(device)
    pred = model(datas)
    t2=time()
    print(f"inf time {t2-t1}")
    loss = criterion(pred, targets)
    loss.backward()
    t3=time()
    print(f"with backward {t3-t1}")
    t1=time()